In [ ]:
import numpy as np
from numpy import numpy_groupies as npg
import scipy.sparse as sps
import numpy.matlib as npm
import cvxpy as cp
import scipy.io
import networkx as nx

## Half Vectorization 


#### README.txt

For a symmetric matrix A, A(:) (full "vectorization") contains more 
information than is strictly necessary, since the matrix is completely 
determined by the symmetry together with the lower triangular portion, 
that is, the n(n+1)/2 entries on and below the main diagonal. The 
half-vectorization, built as following with the package:

A(itril(size(A))), 
of a symmetric n◊n matrix A is the n(n+1)/2 ◊ 1 column vector obtained by 
"vectorizing" only the lower triangular part of A.

This package provides functions for conveniently indexing the triangular
parts (both lower and upper) parts as well as the diagonals of the matrix.

It also provides the so called Duplication and Elimination matrices which
is used to convert between full and half-vectorization of the matrix.

See: http://en.wikipedia.org/wiki/Vectorization_(mathematics)

Please take a look at the script testprog.m to get an idea how the package
works.


## itriu.m

This function returns the subindices for extracting the upper/lower triangular part of a matrix. 

For example 

$A = \begin{bmatrix}
7&5&4\\
4&2&3\\
9&1&9\\
3&5&7
\end{bmatrix}$

then $I = itriu(size(A))$ returns $I = [1,5,6,9,10,11]$ and 

$A(I) = [7,5,2,4,3,9]$.

In [2]:
def itriu(sz, k = 0):
    
    # if isscalar(sz)
        # sz = [sz sz];
    # end
    # m=sz(1);
    # n=sz(2);
    
    # Python implementation
    if np.isscalar(sz):
        sz = [sz, sz]
    m = sz[0]
    n = sz[1]
    
    # Main Diagonal by default
    # if nargin<2
        # k=0;
    # end
    # This is taken care of with default argument k = 0
    
    # nc = n-max(k,0); % number of columns of the triangular part
    
    nc = n - max(k,0)
    
    # lo = ones(nc,1); % lower row indice for each column
    
    lo = [int(i) for i in np.ones(nc)]
    print('lo is ' + str(lo))
    
    # hi = min((1:nc).'-min(k,0),m); % upper row indice for each column
    
    hi = [min(i - min(k,0), m) for i in range(1,nc+1)]
    
    print('hi is ' + str(hi))
    # if isempty(lo)
        # I = zeros(0,1);
        # J = zeros(0,1);
    
    if len(lo) == 0: 
        I = []
        J = []
    
    # else
    
    else:
        
        # c=cumsum([0; hi-lo]+1); % cumsum of the length
        
        temp = [1] + [hi[i] - lo[i] + 1 for i in range(len(hi))]
        c = np.cumsum(temp)
        print('c is ' + str(c))
        
        # I = accumarray(c(1:end-1), (lo-[0; hi(1:end-1)]-1), ... [c(end)-1 1]);
        
        temp = [0] + hi[:len(hi)-1]
        
        print('temp1 is ' + str(temp))
        
        I = npg.aggregate(c[0:len(c)-1], [lo[i] - temp[i] - 1 for i in range(len(lo))], size = c[len(c)-1])[1:]
        
        print('first input is ' + str(c[0:len(c)-1]))
        print('second input is ' + str([lo[i] - temp[i] - 1 for i in range(len(lo))]))
        print('I after aggregating is ' + str(I))
        
        # I = cumsum(I+1); % row indice
        
        I = np.cumsum([i + 1 for i in I])
        
        print('I after cummulation is ' + str(I))
        # J = accumarray(c,1);
        
        J = npg.aggregate(c, 1)[1:]; 
        
        print('J after aggregation is ' + str(J))
        
        # J(1) = 1 + max(k,0); % The row indices starts from this value
        
        J[0] = 1 + max(k,0)
        
        # J = cumsum(J(1:end-1)); % column indice
        
        J = np.cumsum(J[0:len(J)-1])
        
        print('J after cummulation is ' + str(J))
    # end
    
        
        
    # if nargout<2
    # % convert to linear indices
        # I = sub2ind([m n], I, J);
    # end
    
        if k == 0:
            I = [i - 1 for i in I]
            J = [j - 1 for j in J]
            I = np.ravel_multi_index([J,I], [m,n])

    return(I)

In [20]:
itriu([4,4])

lo is [1, 1, 1, 1]
hi is [1, 2, 3, 4]
c is [ 1  2  4  7 11]
temp1 is [0, 1, 2, 3]
first input is [1 2 4 7]
second input is [0, -1, -2, -3]
I after aggregating is [ 0 -1  0 -2  0  0 -3  0  0  0]
I after cummulation is [1 1 2 1 2 3 1 2 3 4]
J after aggregation is [1 1 0 1 0 0 1 0 0 0 1]
J after cummulation is [1 2 2 3 3 3 4 4 4 4]


array([ 0,  4,  5,  8,  9, 10, 12, 13, 14, 15], dtype=int64)

In [3]:
# function [I J] = itril(sz, k)
# function [I J] = itril(sz, k) # OR
# I = itril(sz, k)
#
# Return the subindices [I J] (or linear indices I if single output call)
# in the purpose of extracting an lower triangular part of the matrix of
# the size SZ. Input k is optional shifting. For k=0, extract from the main
# diagonal. For k>0 -> above the diagonal, k<0 -> below the diagonal
# 
# This returnd same as [...] = find(tril(ones(sz),k))
# - Output is a column and sorted with respect to linear indice
# - No intermediate matrix is generated, that could be useful for large
#   size problem
# - Mathematically, A(itril(size(A)) is called (lower) "half-vectorization"
#   of A 
#
# Example:
#
# A = [ 7     5     4
#       4     2     3
#       9     1     9
#       3     5     7 ]
#
# I = itril(size(A))  # gives [1 2 3 4 6 7 8 11 12]'
# A(I)                # gives [7 4 9 3 2 1 5  9  7]' OR A(tril(A)>0)
#
# Author: Bruno Luong <brunoluong@yahoo.com>
# Date: 21/March/2009

# Python implementation of sub2ind found here
# https://stackoverflow.com/questions/28995146/matlab-ind2sub-equivalent-in-python
def sub2ind(array_shape, rows, cols):
    ind = rows*array_shape[1] + cols
    ind[ind < 0] = -1
    ind[ind >= array_shape[0]*array_shape[1]] = -1
    return ind

def itril(sz, k = 0, linear_ind = True):
    
    # if isscalar(sz)
        # sz = [sz sz];
    # end
    # m=sz(1);
    # n=sz(2);
    
    # Python implementation
    if np.isscalar(sz):
        sz = [sz, sz]
    m = sz[0]
    n = sz[1]
    
    # Main Diagonal by default
    # if nargin<2
        # k=0;
    # end
    # This is taken care of with default argument k = 0
    
    # nc = n-max(k,0); % number of columns of the triangular part
    
    nc = n - max(k,0)
    
    # lo = ones(nc,1); % lower row indice for each column
    
    lo = [int(i) for i in np.ones(nc)]
    # print('lo is ' + str(lo))
    
    # hi = min((1:nc).'-min(k,0),m); % upper row indice for each column
    
    hi = [min(i - min(k,0), m) for i in range(1,nc+1)]
    
    # print('hi is ' + str(hi))
    # if isempty(lo)
        # I = zeros(0,1);
        # J = zeros(0,1);
    
    if len(lo) == 0: 
        I = []
        J = []
    
    # else
    
    else:
        
        # c=cumsum([0; hi-lo]+1); % cumsum of the length
        
        temp = [1] + [hi[i] - lo[i] + 1 for i in range(len(hi))]
        c = np.cumsum(temp)
        # print('c is ' + str(c))
        
        # I = accumarray(c(1:end-1), (lo-[0; hi(1:end-1)]-1), ... [c(end)-1 1]);
        
        temp = [0] + hi[:len(hi)-1]
        
        # print('temp1 is ' + str(temp))
        
        I = npg.aggregate(c[0:len(c)-1], [lo[i] - temp[i] - 1 for i in range(len(lo))], size = c[len(c)-1])[1:]
        
        # print('first input is ' + str(c[0:len(c)-1]))
        # print('second input is ' + str([lo[i] - temp[i] - 1 for i in range(len(lo))]))
        # print('I after aggregating is ' + str(I))
        
        # I = cumsum(I+1); % row indice
        
        I = np.cumsum([i + 1 for i in I])
        
        # print('I after cummulation is ' + str(I))
        # J = accumarray(c,1);
        
        J = npg.aggregate(c, 1)[1:]; 
        
        # print('J after aggregation is ' + str(J))
        
        # J(1) = 1 + max(k,0); % The row indices starts from this value
        
        J[0] = 1 + max(k,0)
        
        # J = cumsum(J(1:end-1)); % column indice
        
        J = np.cumsum(J[0:len(J)-1])
        
        # print('J after cummulation is ' + str(J))
    # end
    
        
        
    # if nargout<2
    # % convert to linear indices
        # I = sub2ind([m n], I, J);
    # end
    I = [i - 1 for i in I]
    J = [j - 1 for j in J]
    if linear_ind == True:
        I = np.ravel_multi_index([J,I], [m,n])
        return(I)
    return([J,I])


In [4]:
"""
# function [I J] = idiag(sz, k)
# function [I J] = idiag(sz, k) # OR
# I = itril(sz, k)
#
# Return the subindices [I J] (or linear indices I if single output call)
# in the purpose of extracting the diagonal of the matrix of the size SZ.
# Input k is optional shifting. For k=0, extract from the main
# diagonal. For k>0 -> above the diagonal, k<0 -> below the diagonal
#
# Output is a column and sorted with respect to linear indice
#
# Example:
#
# A = [ 7     5     4
#       4     2     3
#       9     1     9
#       3     5     7 ]
#
# I = idiag(size(A))  # gives [1 6 11]'
# A(I)                # gives [7 2 9]' OR diag(A)
#
# Author: Bruno Luong <brunoluong@yahoo.com>
# Date: 21/March/2009
"""
def idiag(sz, k = 0, linear_ind = True):
    """
    if isscalar(sz)
        sz = [sz sz];
    end
    m=sz(1);
    n=sz(2);
    """
    if np.isscalar(sz):
        sz = [sz, sz]
    m = sz[0]
    n = sz[1]


    """
    # Pay attention to the clipping
    I = (1-min(k,0):min(m,n-k)).';
    J = I+k;
    """
    l = 0 - min(k,0)
    u = min(m,n-k)
    I = list(range(l,u))
    J = [i + k for i in I]
    
    
    """
    if nargout<2
        # convert to linear indices
        I = sub2ind([m n], I, J);
    end
    """
    if linear_ind == True:
        I = np.ravel_multi_index([I,J], [m,n])
        return(I)
    return([I,J])
    # end # idiag


In [5]:
"""
function M = DuplicationM(n, option)
% function M = DuplicationM(n)
% M = DuplicationM(n, 'lo') % (default) OR
% M = DuplicationM(n, 'up') % 
% Return duplication matrix order n
%
% It is always assumed Duplication arount main diagonal (k=0)
%
% Ouput are sparse
%
% DuplicationM(size(A),'lo')*A(itril(size(A))) == A(:) %true for lower half
% DuplicationM(size(A),'up')*A(itriu(size(A))) == A(:) %true for upper half
%
% Author: Bruno Luong <brunoluong@yahoo.com>
% Date: 21/March/2009
%
% Ref : Magnus, Jan R.; Neudecker, Heinz (1980), "The elimination matrix:
% some lemmas and applications", Society for Industrial and Applied Mathematics.
% Journal on Algebraic and Discrete Methods 1 (4): 422449,  
% doi:10.1137/0601049, ISSN 0196-5212.
"""

def DuplicationM(n, option = 'lo'):
    """
    if nargin<2
        option = 'lo'; % default
    end
    """
    
    """ 
    if isscalar(n)
        n = [n n];
    end
    """
    if np.isscalar(n):
        n = [n, n]
    
    """
    switch lower(option(1))
        case 'l' % u, lo, LO, LOWER ...
            [I J] = itril(n);
        case 'u' % u, up, UP, UPPER ...
            [I J] = itriu(n);
        otherwise
            error('option must be ''lo'' or ''up''');
    end
    """
    if option[0].lower() == 'l':
        I, J = itril(n,0,False)
    elif option[0].lower() == 'u':
        J, I = itril(n,0,False)
    else:
        # print("Error, optioin mus be 'lo' or 'up'.")
        return()
    
    I = [x for _, x in sorted(zip(J, I))]
    J = sorted(J)
    """
    % Find the sub/sup diagonal part that can flip to other side
    loctri = find(I~=J & J<=n(1) & I<=n(2));
    """
    # print('I is ' + str(I))
    # print('J is ' + str(J))
    loctri = [i for i in range(len(I)) if 
              (I[i] != J[i]) and 
              (J[i] <= n[0]-1) and 
              (I[i] <= n[1]-1)]
    # print('loctri = ' + str(loctri))
    
    """
    % Indices of the flipped part
    Itransposed = sub2ind(n, J(loctri), I(loctri));
    """
    
    arg1 = [J[i] for i in loctri]
    arg2 = [I[i] for i in loctri]
    
    Itransposed = np.ravel_multi_index([arg1, arg2], n)
    # print('Itransposed = ' + str(Itransposed))
    """
    % Convert to linear indice
    I =  sub2ind(n, I, J);
    """
    I = np.ravel_multi_index([I, J], n)
    
    """
    % Result
    M = sparse([I; Itransposed], ...
               [(1:length(I))'; loctri], 1, prod(n), length(I));
    """
    arg1 = np.append(I,Itransposed)
    arg2 = np.append([i for i in range(len(I))], loctri)
    d = [1]*len(arg1)
    
    # print('length of d is ' + str(len(d)))
    # print('length of I is ' + str(len(I)))
    # print('length of Itransposed is ' + str(len(Itransposed)))
    M = sps.csr_matrix((d, (arg1, arg2)), shape = (np.prod(n),len(I)))
    return(M)

In [24]:
print(DuplicationM(3).toarray())

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [6]:

"""
function [A1,b1,A2,b2,mat_obj] = laplacian_constraint_vech(N)
% constraints        
% mat_cons1*L == zeros(N,1)
% mat_cons2*L <= 0
% vec_cons3*L == N

% %% matrix for constraint 1 (zero row-sum)
% for i = 1:N
%     tmp0{i} = sparse(1,N+1-i);
% end
% 
% mat_cons1 = sparse(N,N*(N+1)/2);
% 
% for i = 1:N
%     
%     tmp = tmp0;
%     tmp{i} = tmp{i}+1;
%     for j = 1:i-1
%         tmp{j}(i+1-j) = 1;
%     end
%     
%     mat_cons1(i,:) = horzcat(tmp{:});
%     
% end
% 
% % for i = 1:N
% %     mat_cons1(i,N*i-N+i-(i*(i-1)/2):N*i-(i*(i-1)/2)) = ones(1,N-i+1);
% % end
% % 
% % for i = 1:N-1
% %     xidx = i+1:N;
% %     yidx = i*(N+N-(i-1))/2-(N-i-1):i*(N+N-(i-1))/2-(N-i-1)+N-i-1;
% %     mat_cons1(sub2ind(size(mat_cons1),xidx,yidx)) = 1;
% % end
% 
% %% matrix for constraint 2 (non-positive off-diagonal entries)
% for i = 1:N
%     tmp{i} = ones(1,N+1-i);
%     tmp{i}(1) = 0;
% end
% 
% mat_cons2 = spdiags(horzcat(tmp{:})',0,N*(N+1)/2,N*(N+1)/2);
% 
% %% vector for constraint 3 (trace constraint)
% vec_cons3 = sparse(ones(1,N*(N+1)/2)-horzcat(tmp{:}));
% 
% %% matrix for objective
% % mat_obj = sparse(N^2,N*(N+1)/2);
% % 
% % for i = 1:N
% %     for j = 1:N
% %         if j <= i-1
% %             tmp = tmp0;
% %             tmp{j}(i+1-j) = 1;
% %             mat_obj((i-1)*N+j,:) = horzcat(tmp{:});
% %         else
% %             tmp = tmp0;
% %             tmp{i}(j-i+1) = 1;
% %             mat_obj((i-1)*N+j,:) = horzcat(tmp{:});
% %         end
% %     end
% % end
% 
% mat_obj = vech2vec(N);
% 
% %% create constraint matrices
% % equality constraint A2*vech(L)==b2
% A1 = [mat_cons1;vec_cons3];
% b1 = [sparse(N,1);N];
% 
% % inequality constraint A1*vech(L)<=b1
% A2 = mat_cons2;
% b2 = sparse(N*(N+1)/2,1);
"""

def laplacian_constraint_vech(N):
    """    
    %% matrix for objective (vech -> vec)
    mat_obj = DuplicationM(N);
    """
    mat_obj = DuplicationM(N)
    
    # Not complete
    """
    X = ones(N);
    [r,c] = size(X);
    i     = 1:numel(X);
    j     = repmat(1:c,r,1);
    B     = sparse(i',j(:),X(:))';
    mat_cons1 = B*mat_obj;
    """
    X = np.ones([N,N])
    r, c = X.shape
    i = range(r*c)
    j = np.matlib.repmat(range(c), r, 1)
    B = sps.csr_matrix((X.flatten('F'), (i, j.flatten('F'))), dtype = np.int_)
    B = B.transpose()
    mat_cons1 = B@mat_obj
    """    
    %% matrix for constraint 2 (non-positive off-diagonal entries)
    for i = 1:N
        tmp{i} = ones(1,N+1-i);
        tmp{i}(1) = 0;
    end
    mat_cons2 = spdiags(horzcat(tmp{:})',0,N*(N+1)/2,N*(N+1)/2);
    """
    Tmp = []
    for i in range(N):
        tmp = [1]*(N-i)
        tmp[0] = 0
        Tmp = Tmp + tmp
    mat_cons2 = sps.spdiags(Tmp, 0, N*(N+1)//2, N*(N+1)//2)
    
    
    """    
    %% vector for constraint 3 (trace constraint)
    vec_cons3 = sparse(ones(1,N*(N+1)/2)-horzcat(tmp{:}));
    """
    
    arg1 = [1 - i for i in Tmp]
    vec_cons3 = sps.csr_matrix(arg1, dtype = np.int_)
    
    
    """    
    %% create constraint matrices
    % equality constraint A2*vech(L)==b2
    A1 = [mat_cons1;vec_cons3];
    b1 = [sparse(N,1);N];
    """
    
    A1 = sps.vstack([mat_cons1, vec_cons3])
    b1 = sps.vstack([sps.csr_matrix((N,1), dtype = np.int_), 
                     sps.csr_matrix(([N], ([0],[0])))])
    
    
    """
    % inequality constraint A1*vech(L)<=b1
    A2 = mat_cons2;
    b2 = sparse(N*(N+1)/2,1);
    """
    
    A2 = mat_cons2;
    b2 = sps.csr_matrix((N*(N+1)//2, 1), dtype = np.int_)
    
    return([A1,b1,A2,b2,mat_obj])
    

In [7]:
N = 3
Tmp = []
for i in range(N):
    tmp = [1]*(N-i)
    tmp[0] = 0
    Tmp = Tmp + tmp
print(Tmp)
mat_cons2 = sps.spdiags(Tmp, 0, N*(N+1)//2, N*(N+1)//2)
print(mat_cons2.toarray())

[0, 1, 1, 0, 1, 0]
[[0 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]]


In [8]:
arg1 = [1 - i for i in Tmp]
vec_cons2 = sps.csr_matrix(arg1)
print(vec_cons2.toarray())

[[1 0 0 1 0 1]]


In [9]:
# Check the inside of function laplacian_constraint_vech
X = np.ones([N,N], dtype = np.int_)
r, c = list(X.shape)
i = range(r*c)
j = npm.repmat(range(c), r, 1)
B = sps.csr_matrix((X.flatten('F'), (i, j.flatten('F'))))
B = B.transpose()
mat_obj = DuplicationM(3)
mat_cons1 = B@mat_obj

In [10]:
arg1 = [1 - i for i in Tmp]
vec_cons3 = sps.csr_matrix(arg1)

In [11]:
sps.vstack([mat_cons1, vec_cons3]).toarray()

array([[1, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 1],
       [1, 0, 0, 1, 0, 1]], dtype=int32)

In [12]:
L = laplacian_constraint_vech(3)
for i in L:
    print(i.toarray())

[[1 1 1 0 0 0]
 [0 1 0 1 1 0]
 [0 0 1 0 1 1]
 [1 0 0 1 0 1]]
[[0]
 [0]
 [0]
 [3]]
[[0 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 0]]
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [13]:
import sys
sys.path.insert(0, './Scripts')
import graph_learning_gaussian as glg

In [14]:
laplacian_constraint_vech(3)

[<4x6 sparse matrix of type '<class 'numpy.intc'>'
 	with 12 stored elements in COOrdinate format>,
 <4x1 sparse matrix of type '<class 'numpy.int32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 <6x6 sparse matrix of type '<class 'numpy.int32'>'
 	with 6 stored elements (1 diagonals) in DIAgonal format>,
 <6x1 sparse matrix of type '<class 'numpy.int32'>'
 	with 0 stored elements in Compressed Sparse Row format>,
 <9x6 sparse matrix of type '<class 'numpy.intc'>'
 	with 9 stored elements in Compressed Sparse Row format>]

In [15]:
"""
function L = optimize_laplacian_gaussian(N,Y,alpha,beta)

%% Laplacian constraints
[A1,b1,A2,b2,mat_obj] = laplacian_constraint_vech(N);
p = vec(Y*Y')';

%% optimization
cvx_begin

% cvx_solver mosek

variable L(N*(N+1)/2,1)

minimize alpha*p*mat_obj*L + beta*sum_square_abs(mat_obj*L)

subject to
    A1*L == b1
    A2*L <= b2

cvx_end

%% convert from vector form to matrix form
% L = convert_matrix(L,N);
L = reshape(mat_obj*L,N,N);

% %% difference vector
% diff = zeros(1,N*(N+1)/2);
% k = 1;
% for i = 1:N
%     for j = i:N
%         diff(k) = -norm(Y(i,:)-Y(j,:),2)^2;
%         k = k+1;
%     end
% end
% 
% %% matrix for constraint 1 (zero row-sum)
% for i = 1:N
%     tmp0{i} = zeros(1,N+1-i);
% end
% 
% mat_cons1 = zeros(N,N*(N+1)/2);
% 
% for i = 1:N
%     
%     tmp = tmp0;
%     tmp{i} = tmp{i}+1;
%     for j = 1:i-1
%         tmp{j}(i+1-j) = 1;
%     end
%     
%     mat_cons1(i,:) = horzcat(tmp{:});
%     
% end
% 
% %% matrix for constraint 2 (non-positive off-diagonal entries)
% for i = 1:N
%     tmp{i} = ones(1,N+1-i);
%     tmp{i}(1) = 0;
% end
% 
% mat_cons2 = diag(horzcat(tmp{:}));
% 
% %% vector for constraint 3 (trace constraint)
% vec_cons3 = ones(1,N*(N+1)/2)-horzcat(tmp{:});
% 
% %% matrix for objective
% mat_obj = zeros(N^2,N*(N+1)/2);
% 
% for i = 1:N
%     for j = 1:N
%         if j <= i-1
%             tmp = tmp0;
%             tmp{j}(i+1-j) = 1;
%             mat_obj((i-1)*N+j,:) = horzcat(tmp{:});
%         else
%             tmp = tmp0;
%             tmp{i}(j-i+1) = 1;
%             mat_obj((i-1)*N+j,:) = horzcat(tmp{:});
%         end
%     end
% end
% 
% %% optimization
% cvx_begin
% 
% % cvx_solver mosek
% 
% variable L(N*(N+1)/2,1)
% 
% minimize alpha*diff*L + beta*sum_square_abs(mat_obj*L)
% 
% subject to
%     mat_cons1*L == zeros(N,1)
%     mat_cons2*L <= 0
%     vec_cons3*L == N
% 
% cvx_end
% 
% %% convert from vector form to matrix form
% L = convert_matrix(L,N);
%
"""
def optimize_laplacian_gaussian(N,Y,alpha,beta):

    """    
    %% Laplacian constraints
    [A1,b1,A2,b2,mat_obj] = laplacian_constraint_vech(N);
    p = vec(Y*Y')';
    """
    A1, b1, A2, b2, mat_obj = laplacian_constraint_vech(N)
    p = (Y@np.transpose(Y)).flatten('F')
    
    L = cp.Variable((N*(N+1)//2, 1))
    constraints = [A1@L == b1,
                   A2@L <= b2]
    objective = cp.Minimize(alpha*(p@mat_obj@L) + beta*cp.sum_squares(mat_obj@L))
    prob = cp.Problem(objective, constraints)
    prob.solve()
    EL = np.reshape(mat_obj@(L.value), (N,N))
    return(EL)
    """    
    %% optimization
    cvx_begin

    % cvx_solver mosek

    variable L(N*(N+1)/2,1)

    minimize alpha*p*mat_obj*L + beta*sum_square_abs(mat_obj*L)

    subject to
        A1*L == b1
        A2*L <= b2

    cvx_end

    %% convert from vector form to matrix form
    % L = convert_matrix(L,N);
    L = reshape(mat_obj*L,N,N);
    """



In [35]:
Y = scipy.io.loadmat('../test_signals/erdos_reyni_signal.mat')['S1']

In [36]:
eL = optimize_laplacian_gaussian(20,Y, .2, .2)

In [37]:
for i in np.transpose(eL)[1]:
    print(i)

9.165131288978207e-20
0.7482585372338355
4.0358268881271134e-20
4.0358191131328286e-20
5.13409995468201e-20
4.024795262337566e-20
4.036176177559943e-20
2.1797124468032158e-20
-0.7482585372338355
4.0250239996171086e-20
4.0251291917431524e-20
2.186672932933086e-20
1.7832100551683615e-20
4.0305631951440985e-20
4.030484864752158e-20
4.0362584618436886e-20
4.037611381346751e-20
4.034244456090144e-20
4.030425074624396e-20
4.0193493062019646e-20


In [16]:
import networkx as nx
"""
function [G, XCoords, YCoords] = construct_graph(N,opt,varargin1,varargin2)
% Graph construction
"""
def construct_graph(N,opt,varagin1):
    
    """
    %% construct the graph
    switch opt
        case 'er', % Erdos-Renyi random graph
            p = varargin1;
            G = erdos_reyni(N,p);

        case 'pa', % scale-free graph with preferential attachment
            m = varargin1;
            G = preferential_attachment_graph(N,m);
    end
    """
    if opt == 'er':
        p = varargin1
        nx.erdos_reyni_graph(N,p)
    else:
        m = varagin1
        G = nx.barabasi_albert_graph(n,m)
    return(G)

In [17]:
"""
function [L,Y,L_harvard] = graph_learning_gaussian(X_noisy,param)
% Learning graphs (Laplacian) from structured signals
% Signals X follow Gaussian assumption
"""
def graph_learning_gaussian(X_noisy, param):
    """
    N = param.N;
    max_iter = param.max_iter;
    alpha = param.alpha;
    beta = param.beta;
    """
    N = param['N']
    max_iter = param['max_iter']
    alpha = param['alpha']
    beta = param['beta']

    """
    objective = zeros(max_iter,1);
    Y_0 = X_noisy;
    Y = Y_0;
    """
    objective = [0]*max_iter
    Y_0 = X_noisy.copy()
    Y = Y_0.copy()     # changed to deep copies 07-04-2022
    
    """
    for i = 1:max_iter

        % Step 1: given Y, update L
        L = optimize_laplacian_gaussian(N,Y,alpha,beta);
        %L = optimize_laplacian_gaussian_admm(N,Y,alpha,beta,0.1,1.5);

        % Step 2: Given L, update Y
        % Y = (eye(N)+alpha*L)^(-1)*Y_0;
        R = chol(eye(N) + alpha*L);
        Y = R \ (R' \ (Y_0));

        % plot the objective
        % objective(i) = norm(Y-Y_0,'fro')^2 + alpha*trace(Y'*L*Y) + beta*(norm(L,'fro')^2);
        objective(i) = norm(Y-Y_0,'fro')^2 + alpha*vec(Y*Y')'*vec(L) + beta*(norm(L,'fro')^2);
        figure(3)
        plot(i,objective(i), '.r');
        hold on, drawnow

        % stopping criteria
        if i>=2 && abs(objective(i)-objective(i-1))<10^(-4)
            break
        end

    end
    """
    
    for i in range(max_iter):
        
        # Step 1: given Y, update L
        L = optimize_laplacian_gaussian(N,Y,alpha,beta)
        
        # Step 2: given L, update Y
        R = np.linalg.cholesky(np.identity(N) + alpha*L)
        Rt = np.transpose(R)
        arg1 = np.linalg.lstsq(Rt, Y_0, rcond=None)[0]   # add rcond=None to suppress the warning message
        print('arg1 shape is ' + str(arg1.shape))
        print('R shape is ' + str(R.shape))
        Y = np.linalg.lstsq(R, arg1, rcond=None)[0]
        
        # Store objective
        arg1 = np.linalg.norm(Y-Y_0, 'fro')**2 
        print("arg1 is: ", arg1)
        arg2 = alpha*(np.transpose((Y@np.transpose(Y)).flatten('F'))@(L.flatten('F')))
        print("arg2 is: ", arg2)
        arg3 = beta*np.linalg.norm(L, 'fro')**2
        print("arg3 is: ", arg3)
        objective[i] = arg1 + arg2 + arg3
        print("Print the objective at the iteration ", i, ": ", objective[i])
        # Stopping criteria
        if i>=2 and abs(objective[i] - objective[i-1]) < 10**(-4):
            break
        
    # return is changd to outside of the for loop, as the final return of the funciton 07-04-2022    
    return([L, Y])

In [18]:
X_noisy = scipy.io.loadmat('../test_signals/X_noisy.mat')['X_noisy']

In [19]:
param = {'N':20, 'max_iter':100, 'alpha':0.0032, 'beta':0.1}

In [20]:
graph_learning_gaussian(X_noisy, param)

arg1 shape is (20, 100)
R shape is (20, 20)
arg1 is:  0.03531371473174675
arg2 is:  7.796886817231165
arg3 is:  3.3757176290676925
Print the objective at the iteration  0 :  11.207918161030605
arg1 shape is (20, 100)
R shape is (20, 20)
arg1 is:  0.03544004038553281
arg2 is:  7.786190194499867
arg3 is:  3.3862235600173265
Print the objective at the iteration  1 :  11.207853794902727
arg1 shape is (20, 100)
R shape is (20, 20)
arg1 is:  0.03544251707310586
arg2 is:  7.785995812093339
arg3 is:  3.3864154407712768
Print the objective at the iteration  2 :  11.20785376993772


[array([[ 1.46212044e+00, -7.74395159e-03, -1.76746542e-22,
         -7.27895509e-23, -4.16915241e-01, -2.49682147e-22,
         -5.08334698e-24, -2.29567638e-01, -1.34703033e-01,
          2.26063438e-23, -1.31594948e-23, -3.20341717e-01,
         -2.61649236e-01, -2.32137875e-23, -9.38315693e-24,
         -3.10669410e-02, -4.14032862e-23, -6.01326846e-02,
         -9.67421800e-24, -8.56259213e-23],
        [-7.74395159e-03,  1.67186902e+00, -1.32654139e-22,
         -1.55177948e-23, -3.39203066e-02, -1.91974041e-22,
         -3.11545767e-23, -2.51128958e-01, -2.61648351e-01,
          4.17566958e-23, -1.11921360e-23, -3.53182877e-01,
         -1.28874837e-01, -7.71021584e-23, -6.78805884e-24,
         -4.92890975e-02, -3.19209284e-01, -2.66871355e-01,
         -3.00671046e-25, -8.32623727e-23],
        [-1.76746542e-22, -1.32654139e-22,  6.99235490e-04,
         -1.38385079e-22, -6.75628362e-23, -3.15248621e-22,
         -1.56465937e-23, -1.12891523e-22, -1.95474794e-22,
         -9.